In [95]:
### Preparing Data
from evaluation import *
import pandas as pd
import random
df2 = pd.read_csv('ner/ner_dataset.csv', encoding= 'unicode_escape')
#df2=df2[["Word","Tag"]]
df_test=df2[["Sentence #"]]
df2=df2.fillna(method='ffill')

In [96]:
Sentence2word = df2.groupby('Sentence #')['Word'].apply(lambda x:x.tolist()).to_dict()
Sentence2tag = df2.groupby('Sentence #')['Tag'].apply(lambda x:x.tolist()).to_dict()

In [97]:
### Data Preprocessing

from collections import defaultdict

def preprocess(data,special_t):  
    tok2idx = defaultdict(lambda: 0)
    tok2idx[special_t] = 0 
    #idx2tok = []
    vocab=list(set(data))
    for i,sp_tok in enumerate(vocab):
        #idx2tok.append(sp_tok)
        if sp_tok not in tok2idx.keys():
            tok2idx[sp_tok]=i+1

    return tok2idx

In [134]:
token2idx = preprocess(df2.Word,'<PAD>')
tag2idx = preprocess(np.unique(df2.Tag)[:-1],'O')


In [129]:
 np.unique(df2.Tag)

array(['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per',
       'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org',
       'I-per', 'I-tim', 'O'], dtype=object)

In [136]:
#idx2token=defaultdict(lambda: 0)
#idx2tag=defaultdict(lambda: 0)
idx2token= [x for x in token2idx]
idx2tag= [x for x in tag2idx]

In [135]:
tag2idx

defaultdict(<function __main__.preprocess.<locals>.<lambda>()>,
            {'O': 0,
             'I-org': 1,
             'B-art': 2,
             'B-org': 3,
             'B-geo': 4,
             'I-eve': 5,
             'B-per': 6,
             'B-eve': 7,
             'I-gpe': 8,
             'I-per': 9,
             'B-gpe': 10,
             'I-geo': 11,
             'I-tim': 12,
             'B-tim': 13,
             'I-art': 14,
             'B-nat': 15,
             'I-nat': 16})

In [100]:
train_size=int(len(Sentence2word.keys())*0.8)
test_size=len(Sentence2word.keys()) - train_size

In [101]:
train_sentence_dict={}
for i in range(train_size):
    
    sentence=random.choice(list(Sentence2word.keys()))
    train_sentence_dict[sentence] = Sentence2word[sentence]
    
train_tags_dict={}    
for sentence in train_sentence_dict.keys():
    
    train_tags_dict[sentence] = Sentence2tag[sentence]

In [102]:
test_sentence_dict={}
test_sentences = [sentence for sentence in Sentence2word.keys() if not sentence in train_sentence_dict.keys()] 
for sentence in test_sentences:
    
    test_sentence_dict[sentence] = Sentence2word[sentence]
    
test_tags_dict={}    
for sentence in test_sentence_dict.keys():
    
    test_tags_dict[sentence] = Sentence2tag[sentence]


In [80]:
test_tags_dict

{'Sentence: 1000': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 'Sentence: 10000': ['B-geo',
  'O',
  'O',
  'B-per',
  'I-per',
  'O',
  'B-tim',
  'O',
  'B-geo',
  'O',
  'B-gpe',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'Sentence: 10006': ['O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-org',
  'I-org',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O',
  'B-per',
  'O',
  'B-per',
  'I-per',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'Sentence: 10010': ['B-gpe',
  'O',
  'O',
  'B-per',
  'I-per',
  'I-per',
  'I-per',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-tim',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'Sentence: 10012': ['O

In [103]:
def batches_generator(batch_size, sentences, tags):
    
    n_samples = len(sentences)
    order = np.arange(n_samples)
    key_list=list(sentences.keys())

    n_batches = n_samples // batch_size
    if n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            
            x_temp = [token2idx[token] for  i,token in enumerate(sentences[key_list[idx]])]
            x_temp = x_temp[:-1]
            x_list.append(x_temp)
            
            y_temp = [tag2idx[tag] for j,tag in enumerate(tags[key_list[idx]])]
            y_temp = y_temp[:-1]
            y_list.append(y_temp)
            
            max_len_token = max(max_len_token, len(tags[key_list[idx]]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            lengths[n] = utt_len
            x[n, :utt_len] = x_list[n]
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

In [104]:
import tensorflow as tf
import numpy as np
tf.compat.v1.disable_eager_execution()

class BiLSTMModel():
    pass

In [105]:
def declare_placeholders(self):
    """Specifies placeholders for the model."""

    # Placeholders for input and ground truth output.
    self.input_batch = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, None], name='label_batch') ######### YOUR CODE HERE #############
  
    # Placeholder for lengths of the sequences.
    self.lengths = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    # Placeholder for a dropout keep probability. If we don't feed
    # a value for this placeholder, it will be equal to 1.0.
    self.dropout_ph = tf.compat.v1.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    # Placeholder for a learning rate (tf.float32).
    self.learning_rate_ph = tf.compat.v1.placeholder(dtype=tf.float32, shape=[]) ######### YOUR CODE HERE #############

In [106]:
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [107]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """Specifies bi-LSTM architecture and computes logits for inputs."""
    
    # Create embedding variable (tf.Variable) with dtype tf.float32
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.compat.v1.Variable(initial_embedding_matrix,dtype=tf.float32,name='embeddings_matrix') ######### YOUR CODE HERE #############
    
    # Create RNN cells (for example, tf.nn.rnn_cell.BasicLSTMCell) with n_hidden_rnn number of units 
    # and dropout (tf.nn.rnn_cell.DropoutWrapper), initializing all *_keep_prob with dropout placeholder.
    forward_cell =  tf.compat.v1.nn.rnn_cell.DropoutWrapper(tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=n_hidden_rnn,use_peepholes=True,name='forward_lstm'),input_keep_prob=self.dropout_ph,output_keep_prob=self.dropout_ph)
    backward_cell =  tf.compat.v1.nn.rnn_cell.DropoutWrapper(tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=n_hidden_rnn,use_peepholes=True,name='back_lstm'),input_keep_prob=self.dropout_ph,output_keep_prob=self.dropout_ph)

    # Look up embeddings for self.input_batch (tf.nn.embedding_lookup).
    # Shape: [batch_size, sequence_len, embedding_dim].
    embeddings = tf.compat.v1.nn.embedding_lookup(embedding_matrix_variable,self.input_batch) ######### YOUR CODE HERE #############
    
    # Pass them through Bidirectional Dynamic RNN (tf.nn.bidirectional_dynamic_rnn).
    # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
    # Also don't forget to initialize sequence_length as self.lengths and dtype as tf.float32.
    (rnn_output_fw, rnn_output_bw), _ = tf.compat.v1.nn.bidirectional_dynamic_rnn(forward_cell,backward_cell,inputs=embeddings,sequence_length=self.lengths,dtype=tf.float32) ######### YOUR CODE HERE #############
    rnn_output = tf.compat.v1.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # Dense layer on top.
    # Shape: [batch_size, sequence_len, n_tags].   
    self.logits = tf.compat.v1.layers.dense(rnn_output, n_tags, activation=None)

In [108]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [109]:
def compute_predictions(self):
    """Transforms logits to probabilities and finds the most probable tags."""
    
    # Create softmax (tf.nn.softmax) function
    softmax_output = tf.nn.softmax(self.logits,name='softmax') ######### YOUR CODE HERE #############
    
    # Use argmax (tf.argmax) to get the most probable tags
    # Don't forget to set axis=-1
    # otherwise argmax will be calculated in a wrong way
    self.predictions = tf.math.argmax(softmax_output,axis=-1) ######### YOUR CODE HERE #############

In [110]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [111]:
def compute_loss(self, n_tags, PAD_index):
    """Computes masked cross-entopy loss with logits."""
    
    # Create cross entropy function function (tf.nn.softmax_cross_entropy_with_logits_v2)
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(ground_truth_tags_one_hot,self.logits) ######### YOUR CODE HERE #############
    
    mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
    # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
    # Be careful that the argument of tf.reduce_mean should be
    # multiplication of mask and loss_tensor.
    self.loss = tf.reduce_mean(loss_tensor*mask) ######### YOUR CODE HERE #############

In [112]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [113]:
def perform_optimization(self):
    """Specifies the optimizer and train_op for the model."""
    
    # Create an optimizer (tf.train.AdamOptimizer)
    self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate_ph) ######### YOUR CODE HERE #############
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
    # Pay attention that you need to apply this operation only for gradients 
    # because self.grads_and_vars also contains variables.
    # list comprehension might be useful in this case.
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars =[ (tf.clip_by_norm(gv[0], clip_norm),gv[1]) for gv in self.grads_and_vars]  ######### YOUR CODE HERE #############
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [114]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [115]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [116]:
BiLSTMModel.__init__ = classmethod(init_model)

In [117]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

In [118]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

In [119]:
def predict_for_batch(self, session, x_batch, lengths):
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

    feed_dict = {self.input_batch: x_batch,
                        self.lengths: lengths}
    predictions = session.run(self.predictions,feed_dict=feed_dict)
    return predictions

In [120]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

In [121]:
tf.compat.v1.reset_default_graph()

model = BiLSTMModel(vocabulary_size=len(token2idx),n_tags=len(tag2idx),embedding_dim=200,n_hidden_rnn=200,PAD_index=token2idx['<PAD>']) ######### YOUR CODE HERE #############

batch_size = 32 ######### YOUR CODE HERE #############
n_epochs = 4  ######### YOUR CODE HERE #############
learning_rate = 0.005 ######### YOUR CODE HERE #############
learning_rate_decay = np.sqrt(2) ######### YOUR CODE HERE #############
dropout_keep_probability = 0.9 ######### YOUR CODE HERE #############

In [143]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())


for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, dict(list(train_sentence_dict.items())[0:20000]), dict(list(train_tags_dict.items())[0:20000]), short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, dict(list(train_sentence_dict.items())[20000:-1]), dict(list(train_tags_dict.items())[20000:-1]), short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size,dict(list(train_sentence_dict.items())[0:20000]) ,dict(list(train_tags_dict.items())[0:20000])):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        #tags_batch, tokens_batch = predict_tags(model, sess, x_batch, lengths)
        #print(tags_batch)
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    

-------------------- Epoch 1 of 4 --------------------
Train data evaluation:
processed 437864 tokens with 46665 phrases; found: 318335 phrases; correct: 2600.

precision:  0.82%; recall:  5.57%; FB1:  1.42


Validation data evaluation:
processed 139324 tokens with 14904 phrases; found: 101180 phrases; correct: 825.

precision:  0.82%; recall:  5.54%; FB1:  1.42


-------------------- Epoch 2 of 4 --------------------
Train data evaluation:
processed 437864 tokens with 46665 phrases; found: 49073 phrases; correct: 24781.

precision:  50.50%; recall:  53.10%; FB1:  51.77


Validation data evaluation:
processed 139324 tokens with 14904 phrases; found: 15345 phrases; correct: 7594.

precision:  49.49%; recall:  50.95%; FB1:  50.21


-------------------- Epoch 3 of 4 --------------------
Train data evaluation:
processed 437864 tokens with 46665 phrases; found: 48383 phrases; correct: 34996.

precision:  72.33%; recall:  74.99%; FB1:  73.64


Validation data evaluation:
processed 139324 tok

In [32]:
from sklearn.metrics import classification_report 

In [36]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx]  )
            tokens.append(idx2token[token_idx] )
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    
    return precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)